In [2]:
#Image size assuming observing parameters, e.g. baseline lengths, sampling, frequency
station_diameter=39
max_baseline_length=70e3
n_pols=4
n_chan=1000
#The 7.5 factor comes from assuming 2xFOV, 3 times sampling of the PSF, and the 1.25 factor in lambda/D
image_size=7.5*(max_baseline_length/station_diameter)**2 * (32/8)
print(image_size)
final_cube= n_pols * n_chan * image_size
print(f'Image size in MBs = {round(final_cube/1e6,5)}')
print(f'Image size in GBs = {round(final_cube/1e9,5)}')
print(f'Image size in TBs = {round(final_cube/1e12,5)}')
print(f'Image size in PBs = {round(final_cube/1e15,5)}')

96646942.80078895
Image size in MBs = 386587.7712
Image size in GBs = 386.58777
Image size in TBs = 0.38659
Image size in PBs = 0.00039


In [173]:
#Assumptions for PSS, DYNAMIC, PST, FLowthrough, and Transient Buffer 
gbps_to_PB=(1/(8000*1e3))
gbps_to_TB=(1/(8000))
GBtoPB=1e6
number_beams=1
number_stations=199
#Assuming a 4 hour observing block
obs_time=4*3600
#---------------------------Flowthrough-----------------------------------------
datarate=1.92 #Gb/s
flowthrough_TB=number_beams*datarate*obs_time*gbps_to_TB
print('Flowthrough=',flowthrough_TB,'TB')

#-------------------------Dynamic spectrum----------------------------------------
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
print('Dynamic spectrum=',dynamic_TB,'TB')

#---------------------------- PST ----------------------------------------------------
datarate=2*1.33 
PST=number_beams*datarate*obs_time*gbps_to_TB
print('PST=',PST,'TB')

#---------------------------- PSS ----------------------------------------------------
scan_size=10 #GB
n_beams=250 #number of pss search beams
pss_TB=8*(n_beams*(scan_size/1e3))
print('PSS=',pss_TB,'TB')

#assuming folder pulse rate 
fp=0.0839 #Gbps
#assuming single pulse is 
sp=0.0209 #Gbps
#8 is assumining max chunk of 30 minutes
pss_TB_updated=16*n_beams*fp*gbps_to_TB*100
print('PSS_updated=',pss_TB_updated,'TB')

#---------------------------Transient Buffer-----------------------------------------

buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams

print('Transient Buffer=',buffersize_4hrs,'TB')


Flowthrough= 3.456 TB
Dynamic spectrum= 3.6864000000000003 TB
PST= 4.788 TB
PSS= 20.0 TB
PSS_updated= 4.195 TB
Transient Buffer= 4.58496 TB


In [3]:
#Number of visibilites
#based on lofar data from Sarrvesh
ant_number=64
n_baselines=(ant_number*(ant_number-1))/2
#n_baselines=1000
obs_t=600
int_time=15
n_chan=5000
n_rows = n_baselines * int( obs_t / int_time)
# What does a single row in an averaged MS contain?
sb_size = n_rows * ((7*8) + \
                         (4+(4*n_chan)) + \
                         (4*11) + \
                         (8*1) + \
                         (4) + \
                         (4 * (8 + 8*n_chan + 4*n_chan)))
# Convert byte length to GB
sb_size /= (1024*1024*1024)

print(round(sb_size,3),'GB')

19.538 GB


In [12]:
#print(buffersize_gbps)
gbps_to_PB=(1/(8000*1e3))
gbps_to_TB=(1/(8000))
def visibility_size_low(number_stations,n_chan,obs_time,int_time,freq_res):
    n_baselines=(number_stations*(number_stations+1))/2
    default_time_res=0.85
    n_rows = n_baselines * int( obs_time / int_time)
    #default_time_res=0.85 
    default_frequency=5.4e3 #default frequency resolution =5.4kHz
    frequency_scaling=freq_res/default_frequency
    n_chan=n_chan/(frequency_scaling)
    #4 polarizations already taken into account.
    sb_size = n_rows * ((7*8) + (4+(4*n_chan)) + (4*11) + (8*1) + (4) + (4 * (8 + 8*n_chan + 4*n_chan)))
    sGB=(1024*1024*1024)
    sTB=(1024*1024*1024*1024)
    sPB=(1024*1024*1024*1024*1024)
    print(f'visib size {round(sb_size/sGB,3)}GB,{round(sb_size/sTB,4)}TB,{round(sb_size/sPB,4)}PB')
    return sb_size/sTB

In [13]:
#print(buffersize_gbps)
gbps_to_PB=(1/(8000*1e3))
gbps_to_TB=(1/(8000))
def image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams):
    image_size=7.5*(max_baseline_length/station_diameter)**2 * (32/8)
    #With residuals+models + a psf
    final_cube= number_beams * n_pols * n_chan_image * image_size *(n_products-1) + (number_beams*image_size) #last part is the PSF which will be 1 channel
    #Just restored images + single channel psf
    #final_cube= (number_beams * n_pols * n_chan_image * image_size) + (number_beams*image_size) #last part is the PSF which will be 1 channel

    sGB=(1024*1024*1024)
    sTB=(1024*1024*1024*1024)
    sPB=(1024*1024*1024*1024*1024)
    print(f'cube size {round(final_cube/sGB,4)}GB, {round(final_cube/sTB,10)}TB,{round(final_cube/sPB,6)}PB')
    totalTB=final_cube/sTB
    return totalTB

In [9]:
#TEST Against file from Sarrvesh to check visibilty size working
station_diameter=39
max_baseline_length=1e3
n_pols,n_products=4,4
#for continuum = 1ximage, 1xresidual, 1*psf (1 channel) 1*model
#for image the psf can be one channel
n_chan_image,n_chan=1,64
number_stations,number_subarray,number_beams=62,1,1
obs_time,int_time=1000,0.85
freq_res=21.6e3
visibility_size_low(number_stations,n_chan,obs_time,int_time,freq_res)
image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

visib size 2.096GB,0.002TB,0.0PB
cube size 0.0002GB, 2.332e-07TB,0.0PB


2.3320376968536626e-07

In [194]:
#YITL LOW 
#---------------------------------- EOR  LOW-KSP-A1  Low_substation_18m_r1km_AAstar Visibilities + image cubes
station_diameter,max_baseline_length,number_stations=39,1e3,796
n_pols,n_products=4,4
n_chan_image=1
number_subarray,number_beams=1,1
n_chan=23962
obs_time=4*3600
int_time=10
freq_res=100e3

#int_time=1
#freq_res=5.4e3
visibility_size_low(number_stations,n_chan,obs_time,int_time,freq_res)
image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

#Commensal project -- KSP-M -- Cont+Transient
# 1 continuum image per hour, 1 Transient dump per hour of 2minutes
print('-----commensal----- KSP-M --')
n_products=4
n_chan_image=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*2,'TB')
#data buffer size
#station number is for AA* within 3kms for buffer data =199
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps

#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours
print(buffersize_4hrs,'TB')
print(buffersize_4hrs+contsize*22)

#print('-----multiplexed commensal---PI-005----')
#Commensal project -- PI-005 -- Cont+Transient
#LOW-PI-005 - low split 8 = 60-73km
# Assuming a continuum image per minute over 4 hours
#Continuum 
station_diameter=39
max_baseline_length=73.4e3
n_pols=4
n_products=4
n_chan_image=1
number_subarray=4
number_beams=4
contsize_pi005=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
# 60 scans per hour over 4 hour SB
#total=60*4*contsize_pi005

#print(round(total,4),'TB')

#total_com=total+buffersize_4hrs+contsize*2
#print('total_comm=',round(total_com,4),'TB')
#removed PI-005 commensal KSP-M, N
print('-----KSP-N---- PSS + Transient---')
n_beams=250
scan_size=10#GB
pss_TB=8*(n_beams*(scan_size/1e3))

kspn=buffersize_4hrs+5
print('KSP-N=',round(kspn,4),'TB')
print(kspn+kspm)

visib size 28686.555GB,28.0142TB,0.0274PB
cube size 0.0002GB, 2.332e-07TB,0.0PB
-----commensal----- KSP-M --
cube size 0.0002GB, 2.332e-07TB,0.0PB
Continnum 4.6640753937073253e-07 TB
18.33984 TB
18.33984513048293
cube size 5.1462GB, 0.0050255972TB,5e-06PB
-----KSP-N---- PSS + Transient---
KSP-N= 23.3398 TB
30.440760784630932


In [9]:
#YITL LOW
#------------------------  LOW-KSP-A2 EoR  Low_substation_18m_r1km_AAstar  Visibilities + image cubes
#image parameters
station_diameter=39
max_baseline_length=1e3
n_pols=4
#for continuum = 1ximage, 1xresidual, 1*psf 1*model
#for image the psf can be one channel
n_products=4
n_chan_image=1
#visibility parameters
number_stations=796
number_subarray=1
number_beams=1
n_chan=14746
number_polarizations=4
obs_time=4*3600
int_time=10
freq_res=100e3
visibility_size_low(number_stations,n_chan,obs_time,int_time,freq_res)
image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

visib size 17677.664GB,17.2633TB,0.0169PB
cube size 0.0002GB, 2.332e-07TB,0.0PB


2.3320376968536626e-07

In [166]:
#YITL LOW
#---------------------------- LOW-KSP-A3 - Low_full_AAstar  Visibilities + image cubes
station_diameter,max_baseline_length=39,73.4e3
n_pols,n_products=4,4
n_chan_image=1

number_stations=307
number_subarray,number_beams=1,1
n_chan=55296
obs_time=4*3600
int_time=4
freq_res=50e3
visibility_size_low(number_stations,n_chan,obs_time,int_time,freq_res)
image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

# 1 continuum image per hour, 1 Transient dump per hour of 2minutes
print('-----commensal---- KSP-M , KSP-Q, ')
print('-----KSP-M---- Cont+Transient---')
n_products=4
n_chan_image=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*2,'TB')
#data buffer size
#station number is for AA* within 3kms for buffer data =199
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps

#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm),'TB')
print('-----KSP-Q-- Cont + Dynamic --')
n_products=4
n_chan_image=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspq=dynamic_TB+2*contsize
print('KSP-Q',round(kspq,4),'TB')


print('-----commensal---Multiplex---- KSP-G , KSP-N, ')
print('-----KSP-G---- Cont+Dynamic---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,68
n_pols,n_products=4,4
n_chan_image=1
number_stations=4
number_subarray,number_beams=1,1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspg=dynamic_TB+2*contsize
print('KSP-G',round(kspg,4),'TB')

print('-----KSP-N---- PSS + Transient---')
n_beams=250
scan_size=10#GB
pss_TB=8*(n_beams*(scan_size/1e3))
pss_TB=5
kspn=buffersize_4hrs+pss_TB
print('KSP-N=',round(kspn,4),'TB')

print('-----Total Commensal---')

print('Total=',round(kspm+kspq+kspg+kspn,4),'TB')

visib size 49248.099GB,48.0938TB,0.047PB
cube size 1.2866GB, 0.0012563993TB,1e-06PB
-----commensal---- KSP-M , KSP-Q, 
-----KSP-M---- Cont+Transient---
cube size 1.2866GB, 0.0012563993TB,1e-06PB
Continnum 0.002512798602812183 TB
7.0732800000000005 TB
KSP-M= 7 TB
-----KSP-Q-- Cont + Dynamic --
cube size 1.2866GB, 0.0012563993TB,1e-06PB
KSP-Q 3.6889 TB
-----commensal---Multiplex---- KSP-G , KSP-N, 
-----KSP-G---- Cont+Dynamic---
cube size 0.0GB, 1.1e-09TB,0.0PB
KSP-G 3.6864 TB
-----KSP-N---- PSS + Transient---
KSP-N= 12.0733 TB
-----Total Commensal---
Total= 26.5495 TB


In [29]:
#YITL LOW
#---------------------------- LOW-KSP-B1, B2, B3, C1, C2-   Cont image + Image cubes
#cont
station_diameter,max_baseline_length=39,73.4e3
number_stations=307
n_pols,n_products=4,4
n_chan_image=1
number_subarray=1
number_beams=3
freq_res=54e3 #factor of 10 sampling for the visibilityes
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#cube
n_chan_image=18432
frequency_sampling=10
n_chan_image=n_chan_image/frequency_sampling
image_cube=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

total=image_cube+contsize*11
print(round(total,4),'TB')

cube size 3.8597GB, 0.0037691979TB,4e-06PB
cube size 6567.1795GB, 6.4132612402TB,0.006263PB
6.4547 TB


In [135]:
#Commensal B1
#KSP-M, P1-0025, KSP-G
#KSP-M, PI0025 Parent array
station_diameter,max_baseline_length=39,73.4e3
number_stations=307
n_pols,n_products=4,4
n_chan_image=1
number_subarray=1
number_beams=3
print('-----KSP-M---- Cont+Transient---')
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*2,'TB')
#data buffer size
#station number is for AA* within 3kms for buffer data =199
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm,4),'TB')

-----KSP-M---- Cont+Transient---
cube size 3.8597GB, 0.0037691979TB,4e-06PB
Continnum 0.007538395808436549 TB
21.21984 TB
KSP-M= 21.3028 TB


In [182]:
#YITL LOW
#---------------------------- LOW-KSP-B1, B2, B3, -   Cont image + Image cubes
#cont
print('-----Commensal------')
print('-----KSP-M---- Cont+Transient---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,73.4e3
number_stations=307
number_subarray=1
number_beams=3
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*22,'TB')
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm,4),'TB')

print('-----PI 25---- Cont')
#Multiplexed
station_diameter=39
max_baseline_length=700
n_pols=4
n_products=4
n_chan_image=1
number_subarray=1
number_beams=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

n_chan_image=11060
n_pols=4
#frequency_sampling=10
#n_chan_image=n_chan_image/10.
image_cube=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

pi025=image_cube+contsize*22
print(round(pi025,4),'TB')
print('-----KSP-G---- Cont+Dynamic---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,68
n_pols,n_products=4,4
n_chan_image=1
number_stations=4
number_subarray,number_beams=1,1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspg=dynamic_TB+2*contsize*22
print('KSP-G',round(kspg,4),'TB')

#print('Total',round(kspg+kspm+pi005,4),'TB')
#print('Total',round(kspg+kspm,4),'TB')


print('-----KSP-N---- PSS + Transient---')
#assuming the transient buffer would be shared with KSP-M. Not assuming now
n_beams=250
scan_size=10#GB
pss_TB=8*(n_beams*(scan_size/1e3))
pss_TB=5
kspn=buffersize_4hrs+pss_TB
pss_TB=5
#kspn=pss_TB
print('KSP-N=',round(kspn,4),'TB')

print('Total',round(kspm+pi025+kspn+kspg,4),'TB')


-----Commensal------
-----KSP-M---- Cont+Transient---
cube size 3.8597GB, 0.0037691979TB,4e-06PB
Continnum 0.08292235389280204 TB
21.21984 TB
KSP-M= 21.3028 TB
-----PI 25---- Cont
cube size 0.0001GB, 1.143e-07TB,0.0PB
cube size 1.1946GB, 0.001166616TB,1e-06PB
0.0012 TB
-----KSP-G---- Cont+Dynamic---
cube size 0.0GB, 1.1e-09TB,0.0PB
KSP-G 3.6864 TB
-----KSP-N---- PSS + Transient---
KSP-N= 26.2198 TB
Total 51.2102 TB


In [183]:
#YITL LOW
#---------------------------- LOW-KSP-C1-   Cont image + Image cubes
#cont
print('-----Commensal------')
print('-----KSP-M---- Cont+Transient---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,73.4e3
number_stations=307
number_subarray=1
number_beams=3
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*22,'TB')
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm,4),'TB')

#print('-----PI 25---- Cont+cubes+ Transient---')
#Assuming this would share the resources of KSP-M?

print('-----KSP-N---- PSS + Transient---')
#assuming the transient buffer would be shared with KSP-M. Not assuming now
n_beams=250
scan_size=10#GB
pss_TB=8*(n_beams*(scan_size/1e3))
pss_TB=5
kspn=buffersize_4hrs+pss_TB
pss_TB=5
#kspn=pss_TB
print('KSP-N=',round(kspn,4),'TB')


print('-----KSP-O---- Zoom---')


contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#cube
#32*1728 !1728 is zoom window number of channels
n_chan_image=55296
#8*1728 !1728 is zoom window number of channels

n_chan_image=8*1728

n_pols=1

image_cube=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

kspo=image_cube+contsize
print('KSP-O',round(kspo,4),'TB')


#Multiplexed
print('-----KSP-G---- Cont+Dynamic---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,68
n_pols,n_products=4,4
n_chan_image=1
number_stations=4
number_subarray,number_beams=1,1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspg=dynamic_TB+2*contsize*22
print('KSP-G',round(kspg,4),'TB')

print('Total',round(kspg+kspm+kspn,4),'TB')


-----Commensal------
-----KSP-M---- Cont+Transient---
cube size 3.8597GB, 0.0037691979TB,4e-06PB
Continnum 0.08292235389280204 TB
21.21984 TB
KSP-M= 21.3028 TB
-----KSP-N---- PSS + Transient---
KSP-N= 26.2198 TB
-----KSP-O---- Zoom---
cube size 3.8597GB, 0.0037691979TB,4e-06PB
cube size 12313.2018GB, 12.0246111294TB,0.011743PB
KSP-O 12.0284 TB
-----KSP-G---- Cont+Dynamic---
cube size 0.0GB, 1.1e-09TB,0.0PB
KSP-G 3.6864 TB
Total 51.209 TB


In [187]:
#YITL LOW
#---------------------------- LOW-KSP-C2/C3 - number beams 3/2 for C2/C3   Cont image + Image cubes
#cont
print('-----Commensal------')
print('-----KSP-M---- Cont+Transient---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,73.4e3
number_stations=307
number_subarray=1
number_beams=2
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*22,'TB')
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm,4),'TB')




print('-----KSP-P----Dynamic ---')
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspp=dynamic_TB+2*contsize*22


print('-----KSP-G---- Cont+Dynamic---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,68
n_pols,n_products=4,4
n_chan_image=1
number_stations=4
number_subarray,number_beams=1,1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspg=dynamic_TB+2*contsize*22
print('KSP-G',round(kspg,4),'TB')

print('-----KSP-I---- PSS ---')
n_beams=250
scan_size=10#GB
pss_TB=8*(n_beams*(scan_size/1e3))
kspi=pss_TB
print('KSP-I',round(kspi,4),'TB')
print('-----KSP-N---- PSS + Transient---')
#assuming the transient buffer would be shared with KSP-M. Not assuming now
n_beams=250
scan_size=10#GB
pss_TB=8*(n_beams*(scan_size/1e3))
pss_TB=5
kspn=buffersize_4hrs+pss_TB
#kspn=pss_TB
print('KSP-N=',round(kspn,4),'TB')

print('Total C2',round(kspg+kspm+kspp,4),'TB')
print('Total C3',round(kspg+kspm+kspi+kspn,4),'TB')


-----Commensal------
-----KSP-M---- Cont+Transient---
cube size 2.5731GB, 0.0025127986TB,2e-06PB
Continnum 0.05528156926186803 TB
14.146560000000001 TB
KSP-M= 14.2018 TB
-----KSP-P----Dynamic ---
-----KSP-G---- Cont+Dynamic---
cube size 0.0GB, 1.1e-09TB,0.0PB
KSP-G 3.6864 TB
-----KSP-I---- PSS ---
KSP-I 20.0 TB
-----KSP-N---- PSS + Transient---
KSP-N= 19.1466 TB
Total C2 25.3716 TB
Total C3 57.0348 TB


In [179]:
#YITL LOW
#---------------------------- LOW-KSP-D -   Cont image + Image cubes
#cont
#10 times /frequency channel sampling
station_diameter=39
max_baseline_length=73.4e3
number_stations=307
n_pols=4
n_products=4
n_chan_image=1
number_subarray=1
number_beams=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#cube
n_chan_image=55296
freq_res=54e3 #factor of 10 sampling for the visibilityes
frequency_sampling=10
n_chan_image=n_chan_image/frequency_sampling

image_cube=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

total=image_cube+contsize*11
print(round(total,4),'TB')
print('commensal ---KSP-M , KSP-O')

print('-----KSP-M---- Cont+Transient---')
n_chan_image=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*22,'TB')
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm,4),'TB')

print('-----KSP-O---- Zoom---')
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#cube
#32*1728 !1728 is zoom window number of channels
n_chan_image=55296
#8*1728 !1728 is zoom window number of channels

n_chan_image=8*1728

n_pols=1

image_cube=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

kspo=image_cube+contsize
print('KSP-O',round(kspo,4),'TB')

print('-----KSP-N---- PSS + Transient---')
#assuming the transient buffer would be shared with KSP-M. Not assuming now
n_beams=250
scan_size=10#GB
pss_TB=8*(n_beams*(scan_size/1e3))
pss_TB=5
kspn=buffersize_4hrs+pss_TB
#kspn=pss_TB
print('KSP-N=',round(kspn,4),'TB')
print('Total',round(kspm+kspo+kspn,4),'TB')


cube size 1.2866GB, 0.0012563993TB,1e-06PB
cube size 6566.9816GB, 6.413067948TB,0.006263PB
6.4269 TB
commensal ---KSP-M , KSP-O
-----KSP-M---- Cont+Transient---
cube size 1.2866GB, 0.0012563993TB,1e-06PB
Continnum 0.027640784630934015 TB
7.0732800000000005 TB
KSP-M= 7.1009 TB
-----KSP-O---- Zoom---
cube size 1.2866GB, 0.0012563993TB,1e-06PB
cube size 4104.4006GB, 4.0082037098TB,0.003914PB
KSP-O 4.0095 23.33984 TB
-----KSP-N---- PSS + Transient---
KSP-N= 12.0733 TB
Total 23.1837 TB


In [200]:
#YITL LOW
#---------------------------- LOW-KSP-E -   Low_full_AAstar Cont image + Image cubes
#cont - want 2 times continuum products at different resolutions (assuming just twice the size, pixel size will probably change slightly)
# Cubes, want 10 times channel sampling and 40' FOV
# At ~200MHz frequency the default field of view is ~160' so 1/4 of the FOV, the current image size in Sarrveshs code is 2FOV so a factor of 8 
station_diameter=39
max_baseline_length=73.4e3
n_pols=4
n_products=4
n_chan_image=1
number_subarray=1
number_beams=1

contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#cube
n_chan_image=45159
frequency_sampling=10
n_chan_image=n_chan_image/10.
image_cube=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

total=image_cube/8+2*contsize*11
n_chan_image=45159
obs_time=4*3600
int_time=4
freq_res=50e3
number_stations=307
visibility_size_low(number_stations,n_chan_image,obs_time,int_time,freq_res)

print(round(total,4),'TB')

print('-----KSP-M---- Cont+Transient---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,73.4e3
number_stations=307
number_subarray=1
number_beams=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*22,'TB')
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm,4),'TB')

print('-----KSP-Q---- Cont+Dynamic---')
n_chan_image=1
number_stations=4
number_subarray,number_beams=1,1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspq=dynamic_TB+2*contsize*22
print('KSP-Q',round(kspq,4),'TB')

print('-----KSP-G---- Cont+Dynamic---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,68
n_pols,n_products=4,4
n_chan_image=1
number_stations=4
number_subarray,number_beams=1,1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspg=dynamic_TB+2*contsize*22
print('KSP-G',round(kspg,4),'TB')


print('Total E',round(kspg+kspm+kspq,4),'TB')



cube size 1.2866GB, 0.0012563993TB,1e-06PB
cube size 5363.1244GB, 5.2374261278TB,0.005115PB
visib size 40224.116GB,39.2814TB,0.0384PB
0.6823 TB
-----KSP-M---- Cont+Transient---
cube size 1.2866GB, 0.0012563993TB,1e-06PB
Continnum 0.027640784630934015 TB
7.0732800000000005 TB
KSP-M= 7.1009 TB
-----KSP-Q---- Cont+Dynamic---
cube size 1.2866GB, 0.0012563993TB,1e-06PB
KSP-Q 3.7417 TB
-----KSP-G---- Cont+Dynamic---
cube size 0.0GB, 1.1e-09TB,0.0PB
KSP-G 3.6864 TB
Total E 14.529 TB


In [203]:
#YITL LOW
#---------------------------- LOW-KSP-F -   Cont image + Image cubes Exoplanets CMI
#Calibrated visibilities with 100 kHz and 10s resolution. 
station_diameter=39
max_baseline_length=73.4e3
n_pols=4
#for continuum = 1ximage, 1xresidual, 1*psf (1 channel) 1*model
#for image the psf can be one channel
n_products=4
n_chan_image=1
number_stations=307
number_subarray=1
number_beams=4
n_chan=55296
obs_time=4*3600
#obs_time=10
int_time=10
freq_res=100e3
n_chan_image=55296
visib=number_beams*visibility_size_low(number_stations,n_chan_image,obs_time,int_time,freq_res)
#estimate image product size based on
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

print('images TB',contsize*11*120*4,'TB')
print('visbility TB',visib,'TB')


print('commensal ---KSP-M , KSP-G, KSP-P')

print('-----KSP-M---- Cont+Transient---')
n_products=4
n_chan_image=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*22,'TB')
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm,4),'TB')


print('-----KSP-P----Dynamic ---')
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspp=dynamic_TB+2*contsize*22

print('-----KSP-G---- Cont+Dynamic---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,68
n_pols,n_products=4,4
n_chan_image=1
number_stations=4
number_subarray,number_beams=1,1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspg=dynamic_TB+2*contsize*22
print('KSP-G',round(kspg,4),'TB')


print('Total F',round(kspg+kspm+kspp,4),'TB')

visib size 9854.312GB,9.6234TB,0.0094PB
cube size 262675.7004GB, 256.5192386603TB,0.250507PB
images TB 1354421.5801262886 TB
visbility TB 38.49340534206479 TB
commensal ---KSP-M , KSP-G, KSP-P
-----KSP-M---- Cont+Transient---
cube size 5.1462GB, 0.0050255972TB,5e-06PB
Continnum 0.11056313852373606 TB
28.293120000000002 TB
KSP-M= 28.4037 TB
-----KSP-P----Dynamic ---
-----KSP-G---- Cont+Dynamic---
cube size 0.0GB, 1.1e-09TB,0.0PB
KSP-G 3.6864 TB
Total F 47.0568 TB


In [25]:
#YITL LOW
#---------------------------- LOW-KSP-G -  Just dynamic spectrum
#Flowthrough mode
datarate=2.048 #Gb/s
obs_time=4*3600

gbps_to_PB=(1/(8000*1e3))
gbps_to_TB=(1/(8000))
GBtoPB=1e6

sb_size_TB=datarate*obs_time*gbps_to_TB
print(round(sb_size_TB,4),'TB')



3.6864 TB


In [196]:
#YITL LOW
#---------------------------- LOW-KSP-H1 -   Cont image + Image cubes RRL Project
#Low_inner_r3km_AAstar
station_diameter=39
max_baseline_length=1.5e3
n_pols=4
n_products=4
n_chan_image=1
number_subarray=1
number_beams=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#cube
#32*1728 !1728 is zoom window number of channels
n_chan_image=55296
n_pols=1

image_cube=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

total=image_cube+contsize*22
print(round(total,4),'TB')


print('commensal ---KSP-M , KSP-G, KSP-P')
station_diameter=39
max_baseline_length=73.4e3
n_pols=4
#for continuum = 1ximage, 1xresidual, 1*psf (1 channel) 1*model
#for image the psf can be one channel
n_products=4
n_chan_image=1
number_stations=307
number_subarray=1
print('-----KSP-M---- Cont+Transient---')
n_products=4
n_chan_image=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*22,'TB')
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm,4),'TB')

print('-----KSP-O---- 8 zooms---')

n_chan_image=19008
n_pols=1

image_cube=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)



print('-----KSP-G---- Cont+Dynamic---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,68
n_pols,n_products=4,4
n_chan_image=1
number_stations=4
number_subarray,number_beams=1,1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspg=dynamic_TB+2*contsize*22
print('KSP-G',round(kspg,4),'TB')


print('Total H1',round(kspg+kspm+kspo,4),'TB')

cube size 0.0005GB, 5.247e-07TB,0.0PB
cube size 6.8563GB, 0.0066956435TB,7e-06PB
0.0067 TB
commensal ---KSP-M , KSP-G, KSP-P
-----KSP-M---- Cont+Transient---
cube size 1.2866GB, 0.0012563993TB,1e-06PB
Continnum 0.027640784630934015 TB
7.0732800000000005 TB
KSP-M= 7.1009 TB
-----KSP-O---- 8 zooms---
cube size 5643.5137GB, 5.5112438587TB,0.005382PB
-----KSP-G---- Cont+Dynamic---
cube size 0.0GB, 1.1e-09TB,0.0PB
KSP-G 3.6864 TB
Total H1 22.8157 TB


In [197]:
#YITL LOW
#---------------------------- LOW-KSP-H2 -   Cont image + Image cubes


station_diameter=39
max_baseline_length=1.5e3
n_pols=4
n_products=4
n_chan_image=1
number_subarray=1
number_beams=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#cube
#11*1728 !1728 is zoom window number of channels
n_chan_image=19008
n_pols=1

image_cube=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

total=image_cube+contsize
print(round(total,4),'TB')

print('commensal ---KSP-M , KSP-G')
station_diameter=39
max_baseline_length=73.4e3
n_pols=4
#for continuum = 1ximage, 1xresidual, 1*psf (1 channel) 1*model
#for image the psf can be one channel
n_products=4
n_chan_image=1
number_stations=307
number_subarray=1
print('-----KSP-M---- Cont+Transient---')
n_products=4
n_chan_image=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*22,'TB')
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm,4),'TB')

print('-----KSP-O---- 8 zooms---')

n_chan_image=19008
n_pols=1

image_cube=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)


print('-----KSP-G---- Cont+Dynamic---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,68
n_pols,n_products=4,4
n_chan_image=1
number_stations=4
number_subarray,number_beams=1,1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspg=dynamic_TB+2*contsize*22
print('KSP-G',round(kspg,4),'TB')

print('-----KSP-O---- 8 zooms---')

print('Total H2',round(kspg+kspm+kspo,4),'TB')

cube size 0.0005GB, 5.247e-07TB,0.0PB
cube size 2.3569GB, 0.0023016539TB,2e-06PB
0.0023 TB
commensal ---KSP-M , KSP-G
-----KSP-M---- Cont+Transient---
cube size 1.2866GB, 0.0012563993TB,1e-06PB
Continnum 0.027640784630934015 TB
7.0732800000000005 TB
KSP-M= 7.1009 TB
-----KSP-O---- 8 zooms---
cube size 5643.5137GB, 5.5112438587TB,0.005382PB
-----KSP-G---- Cont+Dynamic---
cube size 0.0GB, 1.1e-09TB,0.0PB
KSP-G 3.6864 TB
-----KSP-O---- 8 zooms---
Total H2 22.8157 TB


In [104]:
#YITL LOW
#---------------------------- LOW-KSP-I -  PSS
#for PSS 10GB per 30 minute scan
#8 scans per 4 hour window
#250 beams
n_beams=250
scan_size=10#GB
sb_size_TB=8*(n_beams*(scan_size/1e3))
print(round(sb_size_TB,4),'TB')


print('commensal ---KSP-M , KSP-G, PI-023')
station_diameter=39
max_baseline_length=73.4e3
n_pols=4
#for continuum = 1ximage, 1xresidual, 1*psf (1 channel) 1*model
#for image the psf can be one channel
n_products=4
n_chan_image=1
number_stations=307
number_subarray=1
print('-----KSP-M---- Cont+Transient---')
n_products=4
n_chan_image=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*22,'TB')
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm,4),'TB')


print('-----KSP-G---- Cont+Dynamic---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,68
n_pols,n_products=4,4
n_chan_image=1
number_stations=4
number_subarray,number_beams=1,1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspg=dynamic_TB+2*contsize*22
print('KSP-G',round(kspg,4),'TB')

print('-----PI-023---- Cont+Zoom---')

#LOW-PI-23 Low_full_AAstar
#Continuum + zoom
station_diameter=39
max_baseline_length=73.4e3
n_pols=4
n_products=4
n_chan_image=1
number_subarray=1
number_beams=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

#cube 5 *1728 LOW-PI-23
n_chan_image=8640

n_pols=1

image_cube=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

pi023=image_cube+contsize*11

print('Total I',round(kspg+kspm+pi023,4),'TB')

20.0 TB
commensal ---KSP-M , KSP-G, PI-023
-----KSP-M---- Cont+Transient---
cube size 1.2866GB, 0.0012563993TB,1e-06PB
Continnum 0.027640784630934015 TB
7.0732800000000005 TB
KSP-M= 7.1009 TB
-----KSP-G---- Cont+Dynamic---
cube size 0.0GB, 1.1e-09TB,0.0PB
KSP-G 3.6864 TB
-----PI-023---- Cont+Zoom---
cube size 1.2866GB, 0.0012563993TB,1e-06PB
cube size 2565.2875GB, 2.5051635609TB,0.002446PB
Total I 13.3063 TB


In [158]:
#YITL LOW
#---------------------------- LOW-KSP-J -  PST, DYNAMIC, FLOWTHROUGH
#Flowthrough mode
number_beams=3
datarate=1.92 #Gb/s
obs_time=4*3600

gbps_to_PB=(1/(8000*1e3))
gbps_to_TB=(1/(8000))
GBtoPB=1e6
flowthrough_TB=number_beams*datarate*obs_time*gbps_to_TB

datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB

datarate=2*1.33 
PST=number_beams*datarate*obs_time*gbps_to_TB

sb_size_TB=flowthrough_TB+dynamic_TB+PST
print(round(sb_size_TB,4),'TB')


station_diameter=39
max_baseline_length=700
n_pols=4
#for continuum = 1ximage, 1xresidual, 1*psf (1 channel) 1*model
#for image the psf can be one channel
n_products=4
n_chan_image=1
number_stations=240
print('commensal ---KSP-M , KSP-G, KSP-P, KSP-O')

print('-----KSP-M---- Cont+Transient---')
n_products=4
n_chan_image=1
number_beams=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*22,'TB')
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm,4),'TB')


print('-----KSP-O---- Zoom---')
station_diameter=39
max_baseline_length=73.4e3
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#cube
#32*1728 !1728 is zoom window number of channels
n_chan_image=55296
#8*1728 !1728 is zoom window number of channels

n_chan_image=8*1728

n_pols=1

image_cube=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)

kspo=image_cube+contsize*11
print('KSP-O',round(kspo,4),'TB')

print('-----KSP-P----Dynamic ---')
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspp=dynamic_TB+2*contsize*22
print('KSP-P',round(kspp,4),'TB')

print('-----KSP-G---- Cont+Dynamic---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,68
n_pols,n_products=4,4
n_chan_image=1
number_stations=4
number_subarray,number_beams=1,1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspg=dynamic_TB+2*contsize*22
print('KSP-G',round(kspg,4),'TB')

print('Total J',round(kspg+kspm+kspo+kspp,4),'TB')

35.7912 TB
commensal ---KSP-M , KSP-G, KSP-P, KSP-O
-----KSP-M---- Cont+Transient---
cube size 0.0001GB, 1.143e-07TB,0.0PB
Continnum 2.513936637208248e-06 TB
5.529599999999999 TB
KSP-M= 5.5296 TB
-----KSP-O---- Zoom---
cube size 1.2866GB, 0.0012563993TB,1e-06PB
cube size 4104.4006GB, 4.0082037098TB,0.003914PB
KSP-O 4.022 TB
-----KSP-P----Dynamic ---
KSP-P 4.8433 TB
-----KSP-G---- Cont+Dynamic---
cube size 0.0GB, 1.1e-09TB,0.0PB
KSP-G 3.6864 TB
Total J 18.0813 TB


In [206]:
#YITL LOW
#---------------------------- LOW-KSP-k -  PST
#Flowthrough mode
number_beams=8
obs_time=4*3600
number_stations=217
gbps_to_PB=(1/(8000*1e3))
gbps_to_TB=(1/(8000))
GBtoPB=1e6

datarate=2*1.33 
PST=number_beams*datarate*obs_time*gbps_to_TB

sb_size_TB=PST
print(round(sb_size_TB,4),'TB')

print('commensal ---KSP-M , KSP-G, KSP-P, KSP-O')
number_beams=8
print('-----KSP-M---- Cont+Transient---')
station_diameter=39
max_baseline_length=73.4e3
n_products=4
n_chan_image=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*22,'TB')
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps

buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm,4),'TB')

print('-----KSP-L---CONT PSS TRANSIENT ---')

n_products=4
n_chan_image=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*22,'TB')
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams
print(buffersize_4hrs,'TB')

n_beams=250
scan_size=10#GB
pss_TB=8*(n_beams*(scan_size/1e3))
pss_TB=5
kspl=buffersize_4hrs+contsize*11+pss_TB

print('KSP-L=',round(kspl,4),'TB')


print('-----KSP-N---- PSS + Transient---')
#assuming the transient buffer would be shared with KSP-M. Not assuming now
n_beams=250
scan_size=10#GB
pss_TB=8*(n_beams*(scan_size/1e3))
pss_TB=5
kspn=buffersize_4hrs+pss_TB
#kspn=pss_TB
print('KSP-N=',round(kspn,4),'TB')

print('-----KSP-G---- Cont+Dynamic---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,68
n_pols,n_products=4,4
n_chan_image=1
number_stations=4
number_subarray,number_beams=1,1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspg=dynamic_TB+2*contsize*22
print('KSP-G',round(kspg,4),'TB')




print('Total K',round(kspg+kspm+kspn,4),'TB')

38.304 TB
commensal ---KSP-M , KSP-G, KSP-P, KSP-O
-----KSP-M---- Cont+Transient---
cube size 10.2924GB, 0.0100511944TB,1e-05PB
Continnum 0.22112627704747212 TB
39.99744 TB
KSP-M= 40.2186 TB
-----KSP-L---CONT PSS TRANSIENT ---
cube size 10.2924GB, 0.0100511944TB,1e-05PB
Continnum 0.22112627704747212 TB
39.99744 TB
KSP-L= 45.108 TB
-----KSP-N---- PSS + Transient---
KSP-N= 44.9974 TB
-----KSP-G---- Cont+Dynamic---
cube size 0.0GB, 1.1e-09TB,0.0PB
KSP-G 3.6864 TB
Total K 88.9024 TB


In [207]:
#YITL LOW
#---------------------------- LOW-KSP-L -  Buffer + Cont image + Image cube#
#data buffer size
#station number is for AA* within 3kms for buffer data =199
n_stations=199
buffersize_gbps=(192 * 2 * n_stations * (1/1e-6))/1e9 #bps

#print(buffersize_gbps)
gbps_to_PB=(1/(8000*1e3))
gbps_to_TB=(1/(8000))

#assume 2 minutes of data is dumped per hour
#in a 4 hour block
number_hours=4
buffsersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours
print('buffersize',buffsersize_4hrs,'TB')

#print(buffersizeTB,'TB')
n_beams=250
scan_size=10#GB
pss_TB=8*(n_beams*(scan_size/1e3))
pss_TB=5
#Also have continuum
print('pss+buffersize=',pss_TB+buffsersize_4hrs)
station_diameter=39
max_baseline_length=73.4e3
n_pols=4
n_products=4
n_chan_image=1
number_subarray=1
number_beams=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print(contsize)

n_chan_image=55296
freq_res=54e3 #factor of 10 sampling for the visibilityes
frequency_sampling=10
n_chan_image=n_chan_image/frequency_sampling
contsize_image=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print(contsize)

print('Total=',contsize*11+contsize_image)
print('commensal ---KSP-M , KSP-G, KSP-P, KSP-O')

print('-----KSP-M---- Cont+Transient---')
station_diameter=39
max_baseline_length=73.4e3
n_products=4
n_chan_image=1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
print('Continnum',contsize*22,'TB')
buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours
print(buffersize_4hrs,'TB')
kspm=buffersize_4hrs+contsize*22
print('KSP-M=',round(kspm,4),'TB')
print('-----KSP-G---- Cont+Dynamic---')
n_products=4
n_chan_image=1
station_diameter,max_baseline_length=39,68
n_pols,n_products=4,4
n_chan_image=1
number_stations=4
number_subarray,number_beams=1,1
contsize=image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams)
#dynamic
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
kspg=dynamic_TB+2*contsize*22
print('KSP-G',round(kspg,4),'TB')


print('Total K',round(kspg+kspm+kspn,4),'TB')


buffersize 4.58496 TB
pss+buffersize= 9.584959999999999
cube size 1.2866GB, 0.0012563993TB,1e-06PB
0.0012563993014060915
cube size 6566.9816GB, 6.413067948TB,0.006263PB
0.0012563993014060915
Total= 6.426888340312613
commensal ---KSP-M , KSP-G, KSP-P, KSP-O
-----KSP-M---- Cont+Transient---
cube size 1.2866GB, 0.0012563993TB,1e-06PB
Continnum 0.027640784630934015 TB
0.09216 TB
KSP-M= 0.1198 TB
-----KSP-G---- Cont+Dynamic---
cube size 0.0GB, 1.1e-09TB,0.0PB
KSP-G 3.6864 TB
Total K 48.8036 TB


In [7]:
default_time_res=0.85 #default time resolution=0.85
default_frequency=0.54e3 #default frequency resolution =0.54kHz
time_scaling=int_time/default_time_res
frequency_scaling=freq_res/default_frequency

print(time_scaling*frequency_scaling)

2178.6492374727673


In [52]:
#40TB in 60 hours
datasize=90
no_seconds=4*3600
gbps_to_TB=(1/(8000))
datarate=datasize/no_seconds/gbps_to_TB
#frequency resolution
print(round(datarate,3),'gbps')

50.0 gbps


In [ ]:
#Calculate PST
#50